# Setup

In [1]:
# Imports 


import sys
# setting path
sys.path.append('../core')
import os
os.environ["VECLIB_MAXIMUM_THREADS"] = "10"


from sklearn.decomposition import PCA

#bread and butter
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

#machine learning library
import sklearn
import sklearn.manifold
import sklearn.cluster
import sklearn.decomposition

#statistics
import scipy
from scipy.stats import rankdata, norm

import numba
from numba import jit, njit

import tqdm
from tqdm import tqdm
import copy
import cvxpy as cp

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
#import numpy as tnp
import warnings
warnings.filterwarnings('ignore')

# import dfply
# from dfply import *
# import ray
# import datetime


import os
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import scipy

import confound_mdp
import confound_ope
import confound_env
import copy

from subspace import *
from clustering import *
from emalg import *
from helpers import *

%matplotlib inline
plt.style.use('matplotlibrc')
import os
os.environ["VECLIB_MAXIMUM_THREADS"] = "10"

In [2]:
# Generate a single environment


envs = []
# each row:
#   [mdp , pi_b, pi_e, horizon, gamma, nStates, nActions, term]


horizon = 100 #18

#, slip = 0.04, confound_weight=0.6
pi_b, P, R, x_dist, u_dist, gamma = confound_env.gridworld_opetools(horizon = horizon, slip = 0.04, 
                                                                    confound_weight=0.1, small=False, soft=True)

# R = -1*R
gridworld = confound_mdp.ConfoundMDP(P, R, x_dist, u_dist, gamma)

nStates = P.shape[2]
nActions = P.shape[1]

# Generate evaluation policy
pi_e = np.zeros((nStates, nActions))
for i in range(nStates):
    pi_e[i] = [0.4, 0.1, 0.4, 0.1]

    
envs.append([gridworld, pi_b, pi_e, horizon, gamma, nStates, nActions, -1, P, R, x_dist, u_dist, gamma])


In [3]:
# Multiprocessing adjusted for Windows


def getSamplesMultiProc(samples, mdp, pi_b, horizon, iid=True):
    nprocs = multiprocessing.cpu_count()
    with ProcessPoolExecutor(max_workers=nprocs, mp_context=multiprocessing.get_context('spawn')) as executor:
        future = executor.map(collect_sample, [int(samples/nprocs) for i in range(nprocs)], repeat(copy.deepcopy(mdp)), 
                              repeat(copy.deepcopy(pi_b)), repeat(horizon), [i for i in range(nprocs)], repeat(iid))
    dataset = np.vstack(list(future))
    return dataset

In [4]:
# Test behavior policy and obtain its value


for mdp , pi_b, pi_e, base_horizon, gamma, nStates, nActions, term, P, R, x_dist, u_dist, gamma in [envs[-1]]:
    
    print("---")
    
    print("running env with horizon", horizon)

    #dataset = confound_mdp.collect_sample(int(100), mdp, pi_b, horizon, iid=False)
    #pi_b = np.array([pi_b.mean(0),pi_b.mean(0)])
    
    # shape num_traj, horizon, 5 - 5 means (s,a,u,s',r)
    dataset = getSamplesMultiProc(10000, mdp, pi_b, horizon, iid=False)
    
    # shape num_traj*horizon, 5 
    data = dataset.reshape((dataset.shape[0]*dataset.shape[1],5))
    
    # Estimate transition probability
    Phat = confound_ope.estimate_P(dataset, mdp)
    
    # Estimate policy
    pihat = confound_ope.estimate_pi(dataset, mdp)
    
    # Make both 
    for a in range(nActions):
        for s in range(nStates):
            if Phat[a,s].sum() == 0:
                Phat[a,s,term] = 1
            if pihat[s].sum() == 0:
                pihat[s,:] = 1/nActions
                
                
    pi_avg = pi_b[0] * u_dist[0] + pi_b[1] * u_dist[1]

    # This is the actual reward_sa
    # P(s,a,s') * R(s,a,s')
    # R(s,a) = \Sum_{s'} P(s,a,s') * R(s,a,s')
    R_sa = (np.average(P, axis=0, weights=u_dist) * R).sum(axis=2).T #np.sum(R, axis=2).T #.reshape(nStates, nActions)
    P_spsa = Phat.transpose((2, 1, 0))
    n = data.shape[0]


     # behavior value
    print("value of pi_b")
    returns = confound_mdp.calc_returns(dataset, gamma, horizon)
    print(returns.mean())

---
running env with horizon 100
value of pi_b
-4.213761931192736


In [5]:
# Computes the policy, Pb, and (s,a) counts

from numba import njit
#Ptrue_spsa = np.einsum('u,uasp->psa', u_dist, P)

# shape S, A
pi_bsa = np.einsum('u,usa->sa', u_dist, pi_b)

# shape S (s'), S (s), A (a)
Pb_spsa = getPb_spsa(nStates, nActions, u_dist, pi_b, pi_bsa, P)

# starts from a later state (index = burnin) in every trajectory
burnin = 10

# s,a counts for each pair - shape S, A
N_sa = getN_sa(dataset, nStates, nActions, burnin=burnin)

# Subspace Estimation

In [7]:
stateactions = dataset[:, :, [0, 1]]
X = stateactions.reshape(stateactions.shape[0], stateactions.shape[1]*stateactions.shape[2])
fullX = dataset.reshape(dataset.shape[0], dataset.shape[1]*dataset.shape[2])

memorder = 'C'

In [8]:
sadim = nStates*nActions # SA
spdim = nStates # S

# Determine the omega1 and omega2 segments
omegaone = np.array([i for i in range(int(horizon/4), 2*int(horizon/4))])
omegatwo = np.array([i for i in range(3*int(horizon/4), horizon)])

# 
onehotsa = np.eye(int(np.max(stateactions))+1)[stateactions]
onehotsaflat = copy.deepcopy(onehotsa)
onehotsa = onehotsa.reshape((onehotsa.shape[0], onehotsa.shape[1], nStates, nActions), order=memorder)
onehotsp = np.eye(np.max(nextstates)+1)[nextstates]
sz = int(onehotsa.shape[0]/3)

IndexError: arrays used as indices must be of integer (or boolean) type